In [5]:
from langchain_core.prompts import PromptTemplate


* *Prompt Templates*: Um prompt basicamente é uma entrada de texto que fornecemos ao LLM que, por sua vez, processa e nos retona uma resposta. Contudo, quando estamos lidando com LLMs de forma programática, talvez queiramos usar um prompt dinâmico que recebe diferentes parâmetros. Essa é a primeira abstração que nos deparamos ao lidar com LangChain. Para isso temos o *PromptTemplate* que basicamente se trata de uma classe que envolve o prompt e nos permite exeutar nosso prompt várias vezes com difererntes entradas.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

* *Chat models*: O objeto chat model funciona como a nossa interface primária para integarir com LLMs. Essa é a maneira padrão que o LangChain nos ajuda a conversar com os LLMs.

In [2]:
from dotenv import load_dotenv, dotenv_values

# Carregando as variáveis do .env
load_dotenv()
# LangChain puxa automaticamente a variável de ambiente "OPENAI_API_KEY"

True

No execício de SDK standard fizemos assim, lembra?

```py
config = dotenv_values("../.env")
openai_api_key = config["OPENAI_API_KEY"]
```
Aqui estamos apenas carregando a chave e armazenando-a na variável "openai_api_key" mas o LangChain NÃO usa essa variável de maneira direta. Caso quiséssemo utilizá-la teríamos que fazer:
```py
llm = ChatOpenAI(temperature=1.0, model="gpt-4.1-mini", api_key=openai_api_key)
```
### Vamos trabalhar com uma chamada simples. Primeiro passamos um contexto e, segundo esse contexto, vamos dar um comando e esperar uma resposta do LLM

In [3]:
context = """
    Cristiano Ronaldo dos Santos Aveiro é um futebolista português que atua como atacante, mais 
    especificamente como ponta-esquerda ou centroavante, no Al-Nassr e na Seleção Portuguesa, onde é capitão. 
    É o maior goleador da história do futebol em jogos oficiais, sendo simultaneamente o jogador 
    com mais gols na história a nível de seleções bem como a nível de clubes, trajetória que o levou ao recorde 
    de cinco nomeações ao prémio de melhor artilheiro do mundo pela Federação Internacional de
    História e Estatísticas do Futebol (IFFHS). Durante sua passagem pela Europa, se tornou o maior jogador
    histórico e recordista da UEFA Champions League, Nations League e Eurocopa. Também é o único atleta
    a vencer nas três ligas europeias de maior prestígio (Premier League, La Liga e Série A) sendo eleito destaque
    como o melhor jogador e artilheiro das mesmas, quando protagonizava a conquista de seus títulos.
    Cristiano Ronaldo é frequentemente considerado o melhor e mais completo futebolista, bem como o maior
    artilheiro do mundo e na opinião da grande maioria dos especialistas do esporte, seus atributos físicos, 
    suas habilidades goleadoras, sua mentalidade vencedora, sua liderança e seu desempenho sob pressão, 
    o tornam um dos melhores e maiores futebolistas de todos os tempos, com alguns ainda o colocando como o 
    melhor jogador de sempre. Futebolista histórico, é considerado o jogador do século pela Globe Soccer Awards por 
    diversos ex-jogadores, técnicos e dirigentes do futebol. Escalado no Dream Team da Bola de Ouro, 
    também é o jogador com o maior número de nomeações do prémio Ballon d'Or. Foi eleito o melhor jogador 
    do mundo pela FIFA e pela France Football recebendo o prémio Bola de Ouro (que por um período passou a 
    denominar-se Bola de Ouro da FIFA) um total de cinco vezes, sendo o segundo maior vencedor, atrás apenas de 
    Lionel Messi. Também venceu o prémio Bota de Ouro da UEFA num total de quatro vezes. Ganhou uma vez o prémio de 
    Melhor Jogador de Clubes da UEFA. Detém o recorde de três vezes o prémio de Melhor Jogador da UEFA na Europa, 
    três vezes FIFA The Best, seis vezes Globbe Soccer Awards e também o recorde de 15 nomeações na Equipe do Ano da UEFA. 
    Além disso detém quase todos os recordes da Champions League, como a artilharia geral, bem como é amplamente 
    considerado o maior jogador de toda história desta competição.
    """

# Esse placeholder {context} faz referência a variável "context" definida logo acima 
prompt = """    
    Dado o contexto {context} sobre uma pessoa, eu quero que você crie:
    1 - Um pequeno resumo dessa pessoa
    2 - Dois fatos interessantes sobre essa pessoa
    """

In [7]:
prompt_tamplate = PromptTemplate(
    input_variables=["context"],    # Esse ["context"] deve corresponder ao placeholder de prompt
    template=prompt
    )

# Essa linha é o que fará a comunicação com o gpt-4.1 mini e fará as chamadas via API 
llm = ChatOpenAI(temperature=1.0, model="gpt-4.1-mini")

# Esse linha faz o análogo mas utilizando o ollama
edge_llm = ChatOllama(temperature=1.0, model="gemma3:270m") 

# Encadeando o componente da esquerda com o da direita 
chain = prompt_tamplate | llm

# Executando o "chain" acima
response = chain.invoke(input={"context":context})

### Workflow

*1.Você cria um contexto (um texto grande sobre Cristiano Ronaldo)* 

* context = ""..."" 

* É apenas um texto comum em Python. Ele será enviado ao modelo depois.Pense nele como a informação bruta que você quer que o modelo analise 

*2.Você cria um prompt com um placeholder {context}* 

* Esse texto é o template do que você quer perguntar ao modelo. 

* O {context} dentro do prompt é um “buraco” que será preenchido pela variável context 

*3.Você cria o PromptTemplate* 

* Aqui você está dizendo para o LangChain: “O prompt acima tem um espaço chamado context. Sempre que alguém mandar uma variável chamada context, substitua {context} dentro do prompt por esse valor.” Ou seja: O PromptTemplate serve apenas para montar o texto final que irá para o modelo. 

*4.Você cria o LLM* 

* Aqui você cria o modelo de IA que vai receber um texto e gerar uma resposta. Ele não faz nada sozinho ainda. Ele só fica esperando um prompt montado. 

*5.Você cria o Chain* 

* Isso significa: “Pegue o PromptTemplate, use ele para montar o prompt final, depois envie esse prompt para o modelo.” 


* É literalmente uma pipeline: (1) PromptTemplate → (2) LLM (Ou seja: o template organiza o texto, o LLM produz a resposta ) 

*6.response = chain.invoke(input={"context":context})* 

* Aqui ocorre o fluxo completo: 1 - A chain pega a variável context enviada no invoke(); 2 - O PromptTemplate preenche o {context} dentro do prompt; 3 - Esse texto é enviado para o modelo gpt-4.1-mini; 4 - O modelo lê esse prompt pronto e responde; 5 - A resposta final é guardada em response.

